In [1]:
import impmeas as imp
import pandas as pd
import os
from time import time
from tqdm.notebook import tqdm
import sys; sys.setrecursionlimit(15000)
imp.set_pmc_solver(imp.GPMC())

rpath = "./results/ds_iscas99.csv"
fdir = "./cnfs/iscas99"
fnames = [ f"b{str(idx+1).zfill(2)}.bench.dimacs" for idx in range(13) ]
fpaths = [ f"{fdir}/{fname}" for fname in fnames ]

def execute_method(df, cnf_name, method_name, pbar, func):
    if pbar is not None:
        pbar.set_description(f"{cnf_name} / {method_name}")
    else:
        print(f"{cnf_name} / {method_name}")
    t1 = time() 
    ret = func()
    dt = time() - t1
    d = { "benchmark": cnf_name, "method": method_name, "time": dt }
    df = pd.concat((df, pd.DataFrame([d]).set_index("benchmark")))
    return ret, df

def execute_example(df, fpath, methods, pbar):
    fname = fpath.split("/")[-1]
    parsed, cnf, nvars, nclauses = imp.parse_dimacs(open(fpath).read())
    formula = None

    if (m:="GPMC_parsing") in methods: formula, df = execute_method(df, fname, m, pbar, lambda: imp.Formula.parse(parsed))
    if formula is not None:
        if (m:="GPMC_satcount") in methods: _, df = execute_method(df, fname, m, pbar, lambda: imp.GPMC().satcount(cnf))
        if (m:="GPMC_influence") in methods: _, df = execute_method(df, fname, m, pbar, lambda: imp.influence(formula, "x1"))
        if (m:="GPMC_raw_influence") in methods: _, df = execute_method(df, fname, m, pbar, lambda: imp.influence_cnf(cnf, "x1", imp.GPMC()))
        if os.path.getsize(fpath) / 1024 < 1: # the remainder timed out... this too
            if (m:="GPMC_blame") in methods: _, df = execute_method(df, fname, m, pbar, lambda: imp.influence_cnf(cnf, "x1", imp.GPMC()))
    if os.path.getsize(fpath) / 1024 < 10: # only if smaller than 10 kB, the remainder timed out 
        if (m:="BDD_initial_order") in methods: 
            vs, df = execute_method(df, fname, m, pbar, lambda: imp.representation.random_order(cnf))
            imp.buddy_initialize([f"x{idx}" for idx in vs])
            imp.representation.set_dynamic_reordering()
        if (m:="BDD_parsing") in methods: bdd, df = execute_method(df, fname, m, pbar, lambda: imp.BuddyNode.parse(parsed))
        if (m:="BDD_expectation") in methods: _, df = execute_method(df, fname, m, pbar, lambda: bdd.expectation())
        if (m:="BDD_influence") in methods: _, df = execute_method(df, fname, m, pbar, lambda: imp.influence(bdd, "x1"))
        if (m:="BDD_dcgm") in methods: _, df = execute_method(df, fname, m, pbar, lambda: imp.dominating_cgm(bdd))
        if os.path.getsize(fpath) / 1024 < 2: # the remainder timed out
            if (m:="BDD_blame") in methods: _, df = execute_method(df, fname, m, pbar, lambda: imp.blame(bdd, "x1", cutoff=0, debug=False))
    return df

In [ ]:
# comment out methods below to actually execute them (note that there are some dependencies, e.g. "GPMC_influence" and "GPMC_parsing")
methods = [
    # "BDD_initial_order",
    # "BDD_parsing",
    # "BDD_expectation",
    # "BDD_influence",
    # "BDD_dcgm",
    # "BDD_blame",
    # "GPMC_parsing",
    # "GPMC_blame",
    # "GPMC_satcount",
    # "GPMC_influence",
    # "GPMC_raw_influence",
]

df = pd.read_csv(rpath, index_col=["benchmark"]) if os.path.exists(rpath) else pd.DataFrame()
for _ in range(20):
    for fpath in (pbar := tqdm(fpaths)):
        df = execute_example(df, fpath, methods, pbar)
        df.to_csv(rpath)